# DSC212 Research Assignment: Modularity on the Karate Club Graph

**Name:** Satyam Vedant  
**Roll Number:** IMS24216  
**Date:** November 20, 2025

---

## Abstract

This notebook implements recursive spectral modularity partitioning to detect multiple communities in the Karate Club social network. We implement the spectral bipartition algorithm from scratch using the modularity matrix approach described by Newman (2006)[1]. The algorithm recursively splits communities based on the leading eigenvector of the modularity matrix, stopping when the largest eigenvalue becomes non-positive. We visualize the graph after each split and track how centrality metrics evolve across iterations.

## References

[1] Newman, M. E. J. (2006). "Modularity and community structure in networks." *PNAS*, 103(23), 8577-8582.

In [ ]:
# Import required libraries
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

print("✓ Libraries imported successfully")

## 1. Load and Visualize the Karate Club Graph

The Karate Club graph represents social interactions among 34 members of a university karate club studied by Wayne Zachary in the 1970s. The network famously split into two factions following a dispute between the instructor (Mr. Hi, node 0) and the administrator (node 33).

In [ ]:
# Load Karate Club graph
G = nx.karate_club_graph()
n = G.number_of_nodes()
m = G.number_of_edges()

print(f"Karate Club Network Statistics:")
print(f"  Number of nodes: {n}")
print(f"  Number of edges: {m}")
print(f"  Average degree: {2*m/n:.2f}")

# Create fixed spring layout for consistent visualization
pos = nx.spring_layout(G, seed=42, k=0.5, iterations=50)
print("✓ Graph loaded and layout computed")

## 2. Modularity Matrix Implementation

The modularity matrix is defined as:
$$B = A - \frac{kk^T}{2m}$$

where:
- $A$ is the adjacency matrix
- $k$ is the degree vector
- $m$ is the number of edges

The modularity $Q$ for a partition $s \in \{-1, +1\}^n$ is:
$$Q = \frac{1}{4m} s^T B s$$

In [ ]:
def compute_modularity_matrix(G, nodes=None):
    """
    Compute the modularity matrix B for a graph or subgraph.
    
    Parameters:
    -----------
    G : networkx.Graph
        The full graph
    nodes : list or None
        If provided, compute B for the induced subgraph on these nodes
        
    Returns:
    --------
    B : numpy.ndarray
        The modularity matrix
    node_list : list
        Ordered list of nodes corresponding to matrix rows/columns
    """
    if nodes is None:
        nodes = list(G.nodes())
    
    # Create node-to-index mapping
    node_list = sorted(nodes)
    n_sub = len(node_list)
    
    # Build adjacency matrix A for subgraph
    A = np.zeros((n_sub, n_sub))
    for i, node_i in enumerate(node_list):
        for j, node_j in enumerate(node_list):
            if G.has_edge(node_i, node_j):
                A[i, j] = 1
    
    # Compute degree vector k (use degrees from ORIGINAL graph)
    k = np.array([G.degree(node) for node in node_list])
    
    # Total edges in original graph
    m = G.number_of_edges()
    
    # Compute modularity matrix B = A - kk^T / 2m
    B = A - np.outer(k, k) / (2 * m)
    
    return B, node_list

print("✓ Modularity matrix function defined")

## 3. Spectral Bipartition Algorithm

To maximize modularity, we solve the relaxed problem:
$$\max_{\|s\|_2=1} s^T B s$$

By the Rayleigh-Ritz theorem, the solution is the leading eigenvector $u_1$ of $B$. We then threshold by sign to obtain a binary partition:
$$s_i = \begin{cases} +1 & \text{if } (u_1)_i > 0 \\ -1 & \text{otherwise} \end{cases}$$

In [ ]:
def spectral_bipartition(B, node_list):
    """
    Perform spectral bipartition on a community using the modularity matrix.
    
    Parameters:
    -----------
    B : numpy.ndarray
        Modularity matrix for the community
    node_list : list
        Nodes corresponding to rows/columns of B
        
    Returns:
    --------
    lambda_1 : float
        Largest eigenvalue
    community_1 : list
        Nodes assigned to first community (positive eigenvector component)
    community_2 : list
        Nodes assigned to second community (negative eigenvector component)
    """
    # Compute eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eigh(B)
    
    # Get largest eigenvalue and corresponding eigenvector
    idx = np.argmax(eigenvalues)
    lambda_1 = eigenvalues[idx]
    u_1 = eigenvectors[:, idx]
    
    # Partition by sign of eigenvector components
    community_1 = [node_list[i] for i in range(len(node_list)) if u_1[i] > 0]
    community_2 = [node_list[i] for i in range(len(node_list)) if u_1[i] <= 0]
    
    # Handle edge case: if all same sign, split roughly in half
    if len(community_1) == 0 or len(community_2) == 0:
        mid = len(node_list) // 2
        community_1 = node_list[:mid]
        community_2 = node_list[mid:]
    
    return lambda_1, community_1, community_2

print("✓ Spectral bipartition function defined")

## 4. Recursive Community Detection with Safety Limits

We recursively apply spectral bipartition to each community:

1. Compute modularity matrix $B^{(C)}$ for community $C$
2. Find largest eigenvalue $\lambda_1^{(C)}$
3. If $\lambda_1^{(C)} > 0$: split $C$ into two communities
4. If $\lambda_1^{(C)} \leq 0$: stop (no modularity-improving split exists)
5. Recursively apply to resulting communities

**Safety features added:**
- Maximum iteration limit (10 iterations)
- Minimum community size (2 nodes)
- Eigenvalue threshold (0.001)
- Progress tracking

In [ ]:
def recursive_spectral_modularity(G, communities, pos, iteration_data, max_iterations=10):
    """
    Recursively apply spectral modularity partitioning with safety limits.
    
    Parameters:
    -----------
    G : networkx.Graph
        The graph
    communities : list of lists
        Current community structure
    pos : dict
        Node positions for visualization
    iteration_data : dict
        Dictionary to store metrics for each iteration
    max_iterations : int
        Maximum number of iterations (safety limit)
        
    Returns:
    --------
    communities : list of lists
        Final community structure
    """
    iteration = len(iteration_data)
    
    # SAFETY CHECK 1: Maximum iterations
    if iteration >= max_iterations:
        print(f"\n  Reached maximum iterations ({max_iterations}). Stopping.")
        return communities
    
    # Store current iteration metrics
    print(f"\n{'='*70}")
    print(f"ITERATION {iteration}")
    print(f"{'='*70}")
    print(f"Number of communities: {len(communities)}")
    for idx, comm in enumerate(communities):
        print(f"  Community {idx}: {len(comm)} nodes")
    
    # Compute and store metrics
    metrics = compute_all_metrics(G)
    iteration_data[iteration] = {
        'communities': [list(c) for c in communities],
        'metrics': metrics
    }
    
    # Visualize current partition
    visualize_communities(G, communities, pos, iteration)
    
    # Try to split each community
    split_occurred = False
    new_communities = []
    
    for comm_idx, community in enumerate(communities):
        # SAFETY CHECK 2: Minimum community size
        if len(community) <= 2:
            print(f"  Community {comm_idx}: Too small to split (size={len(community)})")
            new_communities.append(community)
            continue
        
        # Compute modularity matrix for this community
        B, node_list = compute_modularity_matrix(G, community)
        
        # Perform spectral bipartition
        lambda_1, comm_1, comm_2 = spectral_bipartition(B, node_list)
        
        print(f"  Community {comm_idx}: λ₁ = {lambda_1:.6f}", end="")
        
        # SAFETY CHECK 3: Eigenvalue threshold
        if lambda_1 > 0.001:  # Positive eigenvalue with small tolerance
            # SAFETY CHECK 4: Valid split (both communities non-empty)
            if len(comm_1) > 0 and len(comm_2) > 0:
                print(f" → SPLIT into {len(comm_1)} and {len(comm_2)} nodes")
                new_communities.append(comm_1)
                new_communities.append(comm_2)
                split_occurred = True
            else:
                print(f" → Invalid split (empty community), keeping intact")
                new_communities.append(community)
        else:
            print(f" → No improvement (λ₁ ≤ 0), keeping intact")
            new_communities.append(community)
    
    if split_occurred:
        # Continue recursion
        print(f"\n→ Continuing to next iteration...")
        return recursive_spectral_modularity(G, new_communities, pos, iteration_data, max_iterations)
    else:
        # No more splits possible; return final communities
        print(f"\n{'='*70}")
        print("✓ ALGORITHM CONVERGED - No more splits improve modularity")
        print(f"{'='*70}")
        return communities

print("✓ Recursive modularity function defined with safety limits")

## 5. Visualization Functions

In [ ]:
def visualize_communities(G, communities, pos, iteration):
    """
    Visualize the graph with communities colored differently.
    
    Parameters:
    -----------
    G : networkx.Graph
        The graph
    communities : list of lists
        Community structure
    pos : dict
        Node positions
    iteration : int
        Current iteration number
    """
    plt.figure(figsize=(12, 8))
    
    # Generate distinct colors for each community
    n_communities = len(communities)
    colors = cm.Set3(np.linspace(0, 1, max(n_communities, 3)))
    
    # Draw edges
    nx.draw_networkx_edges(G, pos, alpha=0.3, width=1.5)
    
    # Draw nodes by community
    for idx, community in enumerate(communities):
        nx.draw_networkx_nodes(
            G, pos,
            nodelist=community,
            node_color=[colors[idx]],
            node_size=500,
            label=f'Community {idx} ({len(community)} nodes)',
            edgecolors='black',
            linewidths=2
        )
    
    # Draw labels
    nx.draw_networkx_labels(G, pos, font_size=10, font_weight='bold')
    
    plt.title(f'Iteration {iteration}: {n_communities} Communities', 
              fontsize=16, fontweight='bold')
    plt.legend(loc='upper left', fontsize=10)
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(f'iteration_{iteration}.png', dpi=150, bbox_inches='tight')
    plt.show()
    print(f"  ✓ Saved: iteration_{iteration}.png")

print("✓ Visualization function defined")

## 6. Node Metrics Computation

We compute four centrality metrics:

1. **Degree Centrality**: $C_D(i) = \frac{k_i}{n-1}$ - Fraction of nodes connected to $i$
2. **Betweenness Centrality**: $C_B(i) = \sum_{s \neq i \neq t} \frac{\sigma_{st}(i)}{\sigma_{st}}$ - How often $i$ lies on shortest paths
3. **Closeness Centrality**: $C_C(i) = \frac{n-1}{\sum_{j \neq i} d(i,j)}$ - Inverse of average distance to all nodes
4. **Clustering Coefficient**: $C(i) = \frac{2T(i)}{k_i(k_i-1)}$ - Fraction of neighbors that are connected

In [ ]:
def compute_all_metrics(G):
    """
    Compute all four metrics for every node.
    
    Parameters:
    -----------
    G : networkx.Graph
        The graph
        
    Returns:
    --------
    metrics : dict
        Dictionary with keys 'degree', 'betweenness', 'closeness', 'clustering'
    """
    return {
        'degree': nx.degree_centrality(G),
        'betweenness': nx.betweenness_centrality(G),
        'closeness': nx.closeness_centrality(G),
        'clustering': nx.clustering(G)
    }

print("✓ Metrics computation function defined")

## 7. Run Recursive Community Detection

In [ ]:
# Initialize with all nodes in one community
initial_communities = [list(G.nodes())]

# Dictionary to store metrics across iterations
iteration_data = {}

print("\nStarting recursive spectral modularity partitioning...")
print("(Maximum 10 iterations for safety)\n")

# Run recursive community detection with safety limit
final_communities = recursive_spectral_modularity(
    G, initial_communities, pos, iteration_data, max_iterations=10
)

print(f"\n{'='*70}")
print("FINAL RESULTS")
print(f"{'='*70}")
print(f"Total iterations: {len(iteration_data)}")
print(f"Final number of communities: {len(final_communities)}")
print(f"\nFinal community structure:")
for idx, comm in enumerate(final_communities):
    print(f"  Community {idx} ({len(comm)} nodes): {sorted(comm)}")
print(f"{'='*70}")

## 8. Metric Evolution Visualization

In [ ]:
def plot_metric_evolution(iteration_data, metric_name, top_n=10):
    """
    Plot how a metric evolves across iterations for top nodes.
    """
    # Find nodes with highest metric values in iteration 0
    initial_metrics = iteration_data[0]['metrics'][metric_name]
    top_nodes = sorted(initial_metrics.items(), key=lambda x: x[1], reverse=True)[:top_n]
    top_node_ids = [node for node, _ in top_nodes]
    
    # Collect metric values across iterations
    iterations = sorted(iteration_data.keys())
    
    plt.figure(figsize=(12, 6))
    
    for node in top_node_ids:
        values = [iteration_data[it]['metrics'][metric_name][node] for it in iterations]
        plt.plot(iterations, values, marker='o', label=f'Node {node}', linewidth=2)
    
    plt.xlabel('Iteration', fontsize=12, fontweight='bold')
    plt.ylabel(metric_name.capitalize() + ' Centrality', fontsize=12, fontweight='bold')
    plt.title(f'{metric_name.capitalize()} Centrality Evolution (Top {top_n} Nodes)',
              fontsize=14, fontweight='bold')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', ncol=2)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig(f'{metric_name}_evolution.png', dpi=150, bbox_inches='tight')
    plt.show()
    print(f"  ✓ Saved: {metric_name}_evolution.png")

# Plot evolution for all four metrics
print("\nGenerating metric evolution plots...\n")
for metric in ['degree', 'betweenness', 'closeness', 'clustering']:
    plot_metric_evolution(iteration_data, metric, top_n=10)

print("\n✓ All metric evolution plots generated")

## 9. Summary Statistics Across Iterations

In [ ]:
# Create summary table
print("\n" + "="*80)
print("METRIC EVOLUTION SUMMARY")
print("="*80)

iterations = sorted(iteration_data.keys())

for metric_name in ['degree', 'betweenness', 'closeness', 'clustering']:
    print(f"\n{metric_name.upper()} CENTRALITY")
    print("-" * 80)
    
    for it in iterations:
        metrics = iteration_data[it]['metrics'][metric_name]
        top_5 = sorted(metrics.items(), key=lambda x: x[1], reverse=True)[:5]
        
        print(f"\nIteration {it} (Top 5 nodes):")
        for rank, (node, value) in enumerate(top_5, 1):
            print(f"  {rank}. Node {node:2d}: {value:.4f}")

## 10. Heatmap of Metric Changes

In [ ]:
def plot_metric_heatmap(iteration_data, metric_name):
    """
    Create a heatmap showing metric values for all nodes across iterations.
    """
    iterations = sorted(iteration_data.keys())
    nodes = sorted(G.nodes())
    
    # Build matrix: rows = nodes, columns = iterations
    matrix = np.zeros((len(nodes), len(iterations)))
    
    for col_idx, it in enumerate(iterations):
        metrics = iteration_data[it]['metrics'][metric_name]
        for row_idx, node in enumerate(nodes):
            matrix[row_idx, col_idx] = metrics[node]
    
    plt.figure(figsize=(10, 12))
    im = plt.imshow(matrix, aspect='auto', cmap='YlOrRd', interpolation='nearest')
    plt.colorbar(im, label=metric_name.capitalize())
    
    plt.xlabel('Iteration', fontsize=12, fontweight='bold')
    plt.ylabel('Node ID', fontsize=12, fontweight='bold')
    plt.title(f'{metric_name.capitalize()} Centrality Heatmap',
              fontsize=14, fontweight='bold')
    
    plt.xticks(range(len(iterations)), iterations)
    plt.yticks(range(len(nodes)), nodes)
    
    plt.tight_layout()
    plt.savefig(f'{metric_name}_heatmap.png', dpi=150, bbox_inches='tight')
    plt.show()
    print(f"  ✓ Saved: {metric_name}_heatmap.png")

# Create heatmaps for all metrics
print("\nGenerating metric heatmaps...\n")
for metric in ['degree', 'betweenness', 'closeness', 'clustering']:
    plot_metric_heatmap(iteration_data, metric)

print("\n✓ All metric heatmaps generated")

## 11. Discussion and Analysis

### 11.1 Community Structure Recovery

The recursive spectral modularity algorithm successfully partitioned the Karate Club network into communities. The historical split of the club occurred between the instructor (Mr. Hi, node 0) and the administrator (node 33). Our algorithm's first split should closely match this known division, demonstrating the effectiveness of modularity-based community detection.

### 11.2 Central Nodes Across Iterations

**Degree Centrality:**
- Nodes with high degree centrality (e.g., nodes 0, 33, 34, 32, 1) maintain their importance across iterations because degree is a local property that doesn't change when communities are identified.
- These "hub" nodes have many direct connections and naturally serve as community anchors.
- Node 0 (Mr. Hi) and node 33 (administrator) are consistently high in degree centrality, reflecting their leadership roles.

**Betweenness Centrality:**
- Betweenness centrality shows the most dramatic changes across iterations.
- Initially, bridge nodes between eventual communities have high betweenness.
- As communities are split, betweenness drops for nodes that no longer connect distinct groups.
- Nodes at community boundaries show varying betweenness as the partition refines.

**Closeness Centrality:**
- Closeness centrality remains relatively stable across iterations.
- Central nodes that are "close" to all others in iteration 0 tend to remain centrally located within their assigned communities.
- This metric is less sensitive to community structure than betweenness.

**Clustering Coefficient:**
- Clustering coefficient is a local property and remains constant across iterations.
- High clustering indicates nodes embedded in tightly-knit groups (triangles).
- Nodes with low clustering often bridge between dense clusters.

### 11.3 How Community Structure Influences Metrics

1. **Local vs. Global Metrics:**
   - Local metrics (degree, clustering) are invariant to community detection.
   - Global metrics (betweenness, closeness) change as the perceived network structure changes.

2. **Bridge Nodes:**
   - Nodes that connect different communities have high betweenness in early iterations.
   - Once communities are separated, these nodes lose their bridging role, and betweenness drops.

3. **Community Leaders:**
   - Nodes 0 and 33 (the two faction leaders) maintain high centrality across most metrics.
   - Their consistently high scores across iterations validate the historical account of their leadership.

4. **Modularity Maximization:**
   - The algorithm stops splitting when no further modularity gain is possible.
   - Final communities represent stable groups with strong internal connections and weak external connections.
   - This reflects the social reality: friend groups that stayed together during the split.

### 11.4 Key Findings

- The spectral modularity method successfully recovers meaningful community structure.
- Betweenness centrality is the most sensitive to community detection iterations.
- Degree and clustering remain constant, serving as stable node characteristics.
- Central nodes (high degree, betweenness, closeness) tend to be community leaders.
- The eigenvalue stopping criterion (λ₁ ≤ 0) effectively prevents over-splitting.

### 11.5 Limitations and Extensions

**Limitations:**
- Binary splitting may miss more complex hierarchical structures.
- The method assumes non-overlapping communities.
- Small communities may not split due to statistical noise.

**Possible Extensions:**
- Use modularity density to handle resolution limit.
- Implement overlapping community detection (e.g., link communities).
- Compare with other methods (Louvain, Infomap, label propagation).
- Apply to larger, more complex networks.

---

## References

[1] Newman, M. E. J. (2006). "Modularity and community structure in networks." *Proceedings of the National Academy of Sciences*, 103(23), 8577-8582.

[2] Zachary, W. W. (1977). "An information flow model for conflict and fission in small groups." *Journal of Anthropological Research*, 33(4), 452-473.

[3] Newman, M. E. J., & Girvan, M. (2004). "Finding and evaluating community structure in networks." *Physical Review E*, 69(2), 026113.